# Public opinion and sentiment analysis on the COVID-19 vaccine using Twitter data

* **Team**: Data Geeks
* **Team Members**: 
Sanjay Bhargav Madamanchi
Chirudeep Gorle
Raparla Sravani
TanujaReddy Maligireddy

# Connection With Drive

In [1]:
#cannot share in github right now as the repository is private.
from google.colab import drive
drive.mount('/content/gdrive')
# Mount Google Drive
# from google.colab import drive
# import pandas as pd
# drive.mount('/content/drive')

Mounted at /content/gdrive


In [2]:
import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/CMPE255_Project/Data"

In [5]:
!unzip \*.zip  && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


# Import packages

In [6]:
!pip install neattext
import neattext as ntx

     |████████████████████████████████| 114 kB 4.1 MB/s 


In [7]:
# Import standard libraries
import numpy as np
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm, dates
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import string, collections, unicodedata
import missingno as msno

# Import NLP libraries
import re
import nltk
import json
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import subjectivity
%matplotlib inline 

# Import models, evaluator libraries
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix

# Import location, time libraries
import os
import time
from datetime import datetime, timedelta

%matplotlib inline

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning:

The twython library has not been installed. Some functionality from the twitter package will not be available.



In [8]:
!pip install langdetect


     |████████████████████████████████| 981 kB 4.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=8f374864dab2649695d34c16cc74bd955c5dae43e6b7edee92423647b04b6ce5
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [9]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 6.3 MB 43.0 MB/s 
     |████████████████████████████████| 15.4 MB 41.2 MB/s 


In [10]:
pip install -U textblob

     |████████████████████████████████| 636 kB 4.4 MB/s 
  Attempting uninstall: textblob
    Found existing installation: textblob 0.15.3
    Uninstalling textblob-0.15.3:
      Successfully uninstalled textblob-0.15.3


In [11]:
#Working with Data/CSV of movies with Panda and Numpy libraries
import pandas as pd
import numpy as np

from pandas import Series, DataFrame
pd.options.mode.chained_assignment = None



In [12]:
from textblob import TextBlob

b = TextBlob('So excited to get my vaccine!')
print(b.sentiment)

Sentiment(polarity=0.46875, subjectivity=0.75)


# Twitter Data: Load Data

* https://www.kaggle.com/gpreda/all-covid19-vaccines-tweets
* https://www.kaggle.com/gpreda/covid-world-vaccination-progress


In [ ]:
#Reading vaccination_all_tweets CSV and Loading our data frame

df_vacc_tweet = pd.read_csv('/content/gdrive/MyDrive/CMPE255_Project/Data/vaccination_all_tweets.csv', lineterminator='\n', parse_dates=True)
df_tweets_training = pd.read_csv('./tweet_dataset.csv')
df_tweets_training.head(3)

In [ ]:
df_vacc_tweet.shape

In [ ]:

df_vacc_tweet.columns

In [ ]:
df_vacc_tweet.sort_values(by='retweets', ascending=False)[['text', 'date', 'user_name', 'user_location', 'hashtags', 'favorites', 'retweets']].head(n=10)

In [ ]:
df_vacc_tweet.sort_values(by=['date', 'favorites'], ascending=[True, False])[['text', 'date', 'user_name', 'user_location', 'hashtags', 'favorites', 'retweets']].head(n=10)

In [ ]:
# df_vacc_tweet['polarity'] = df_vacc_tweet['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
# df_vacc_tweet['subjectivity'] = df_vacc_tweet['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_vacc_tweet.head()

# Understanding & Preprocessing Data

## Data Info

In [ ]:
df_vacc_tweet.info()

In [ ]:
df_vacc_tweet.describe()

## Missing Data Check

- We can see that there is a lot of missing data in user_location, description,sources

In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
missing_data(df_vacc_tweet)

In [ ]:
df_vacc_tweet.isna().sum()

## Unique Values

In [ ]:
def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique()
        uniques.append(unique)
    tt['Uniques'] = uniques
    return(np.transpose(tt))

In [ ]:
unique_values(df_vacc_tweet)

## Most Frequent Values

In [ ]:
def most_frequent_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    items = []
    vals = []
    for col in data.columns:
        itm = data[col].value_counts().index[0]
        val = data[col].value_counts().values[0]
        items.append(itm)
        vals.append(val)
    tt['Most frequent item'] = items
    tt['Frequence'] = vals
    tt['Percent from total'] = np.round(vals / total * 100, 3)
    return(np.transpose(tt))

In [ ]:
most_frequent_values(df_vacc_tweet)

## Converting date column to date format

In [ ]:

df_vacc_tweet['date_new'] = pd.to_datetime(df_vacc_tweet['date']).dt.date  #converting date column to date format
df_vacc_tweet.head()

## Dropping Duplicate Tweets

In [ ]:
df_vacc_tweet=df_vacc_tweet.drop_duplicates('text')             
df_vacc_tweet.shape

In [ ]:
df_vacc_tweet.source.value_counts()

## Date: Number of days considered

In [ ]:
len(df_vacc_tweet['date'].unique())  

In [ ]:
df_vacc_tweet.sort_values(by=['date'], ascending=[True]).head(2)

## Dropping Unnecessary Columns- will decide later

In [ ]:
# data1 = df_vacc_tweet.drop(columns={"id","user_name","user_description","user_created","user_followers",\
#                    "user_friends","user_favourites","user_verified","hashtags","source","retweets","favorites","is_retweet\r"},inplace=True)
# # dropping unnecessary 

## Set Data Max Width

In [ ]:
pd.set_option('display.max_colwidth', 700)
df_vacc_tweet.head()

# Visualize Data Distribution

## Visualizing Tweet Platform-wise Distribution 

In [ ]:

plt.figure(figsize=(15,10))
df_vacc_tweet['source'].value_counts().nlargest(6).plot(kind='bar')
plt.xticks(rotation=80)

## User Name

In [ ]:
def plot_count(feature, title, df, size=1, ordered=True):
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(df))
    if ordered:
        g = sns.countplot(df[feature], order = df[feature].value_counts().index[:20], palette='Set3')
    else:
        g = sns.countplot(df[feature], palette='Set3')
    g.set_title("Number and percentage of {}".format(title))
    if(size > 2):
        plt.xticks(rotation=90, size=8)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show()  

In [ ]:
plot_count("user_name", "User name", df_vacc_tweet,4)

## User Location

In [ ]:
plot_count("user_location", "User location", df_vacc_tweet,4)

## Tweet Source

In [ ]:
plot_count("source", "Source", df_vacc_tweet,4)

## Visulizing Tweet Count vs Location

In [ ]:
  
plt.figure(figsize=(15,10))
df_vacc_tweet['user_location'].value_counts().nlargest(20).plot(kind='bar')
plt.xticks(rotation=60)

## Text WordClouds

In [ ]:
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=50,
        max_font_size=40, 
        scale=5,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(10,10))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
def show_wordcloud(data, title=""):
    text = " ".join(t for t in data.dropna())
    stopwords = set(STOPWORDS)
    stopwords.update(["t", "co", "https", "amp", "U"])
    wordcloud = WordCloud(stopwords=stopwords, scale=4, max_font_size=50, max_words=500,background_color="black").generate(text)
    fig = plt.figure(1, figsize=(16,16))
    plt.axis('off')
    fig.suptitle(title, fontsize=20)
    fig.subplots_adjust(top=2.3)
    plt.imshow(wordcloud, interpolation='bilinear')

In [ ]:
df_vacc_tweet['user_location'].unique()

### Prevalent words in tweets

#### 1. All

In [ ]:
show_wordcloud(df_vacc_tweet['text'], title = 'Prevalent words in tweets')

#### 2. USA

In [ ]:
us_df = df_vacc_tweet.loc[df_vacc_tweet.user_location=="United States"]
show_wordcloud(us_df['text'], title = 'Prevalent words in tweets from US')

#### 3. California

In [ ]:
ca_df = df_vacc_tweet.loc[df_vacc_tweet.user_location=="California, USA"]
show_wordcloud(ca_df['text'], title = 'Prevalent words in tweets from CA, USA')

#### 4. UK

In [ ]:
uk_df = df_vacc_tweet.loc[df_vacc_tweet.user_location=="United Kingdom"]
show_wordcloud(uk_df['text'], title = 'Prevalent words in tweets from UK')

#### 5. Canada

In [ ]:
ca_df = df_vacc_tweet.loc[df_vacc_tweet.user_location=="Canada"]
show_wordcloud(ca_df['text'], title = 'Prevalent words in tweets from Canada')

#### 6. India

In [ ]:
india_df = df_vacc_tweet.loc[df_vacc_tweet.user_location=="India" ]
show_wordcloud(india_df['text'], title = 'Prevalent words in tweets from India')

#### 7. Bangladesh

In [ ]:
bd_df = df_vacc_tweet.loc[df_vacc_tweet.user_location=="Bangladesh"]
show_wordcloud(bd_df['text'], title = 'Prevalent words in tweets from Bangladesh')

# Hashtag Analysis

In [ ]:
def plot_features_distribution(features, title, df, isLog=False):
    plt.figure(figsize=(12,6))
    plt.title(title)
    for feature in features:
        if(isLog):
            sns.distplot(np.log1p(df[feature]),kde=True,hist=False, bins=120, label=feature)
        else:
            sns.distplot(df[feature],kde=True,hist=False, bins=120, label=feature)
    plt.xlabel('')
    plt.legend()
    plt.show()

In [ ]:
df_vacc_tweet['hashtags'] = df_vacc_tweet['hashtags'].replace(np.nan, "['None']", regex=True)
df_vacc_tweet['hashtags'] = df_vacc_tweet['hashtags'].apply(lambda x: x.replace('\\N',''))
df_vacc_tweet['hashtags_count'] = df_vacc_tweet['hashtags'].apply(lambda x: len(x.split(',')))
plot_features_distribution(['hashtags_count'], 'Hashtags per tweet (all data)', df_vacc_tweet)

In [ ]:
df_vacc_tweet['hashtags_individual'] = df_vacc_tweet['hashtags'].apply(lambda x: x.split(','))
from itertools import chain
all_hashtags = set(chain.from_iterable(list(df_vacc_tweet['hashtags_individual'])))
print(f"There are totally: {len(all_hashtags)}")

In [ ]:
df_vacc_tweet['hashtags_individual'].head()

# Date and Time Feature Analysis

- Time Variation

In [ ]:
tweets_df = df_vacc_tweet
tweets_df['datedt'] = pd.to_datetime(tweets_df['date'])

In [ ]:
tweets_df['year'] = tweets_df['datedt'].dt.year
tweets_df['month'] = tweets_df['datedt'].dt.month
tweets_df['day'] = tweets_df['datedt'].dt.day
tweets_df['dayofweek'] = tweets_df['datedt'].dt.dayofweek
tweets_df['hour'] = tweets_df['datedt'].dt.hour
tweets_df['minute'] = tweets_df['datedt'].dt.minute
tweets_df['dayofyear'] = tweets_df['datedt'].dt.dayofyear
tweets_df['date_only'] = tweets_df['datedt'].dt.date

In [ ]:
tweets_agg_df = tweets_df.groupby(["date_only"])["text"].count().reset_index()
tweets_agg_df.columns = ["date_only", "count"]

In [ ]:
def plot_time_variation(df, x='date_only', y='count', hue=None, size=1, title="", is_log=False):
    f, ax = plt.subplots(1,1, figsize=(4*size,3*size))
    g = sns.lineplot(x=x, y=y, hue=hue, data=df)
    plt.xticks(rotation=90)
    if hue:
        plt.title(f'{y} grouped by {hue} | {title}')
    else:
        plt.title(f'{y} | {title}')
    if(is_log):
        ax.set(yscale="log")
    ax.grid(color='black', linestyle='dotted', linewidth=0.75)
    plt.show() 

## Plot: Number of tweets / day of year

In [ ]:
plot_time_variation(tweets_agg_df, title="Number of tweets / day of year",size=3)

## Pkot: Tweets / day of week

In [ ]:
plot_count("dayofweek", "tweets / day of week", tweets_df, size=3, ordered=False)

## Plot: Tweets / day of year

In [ ]:
plot_count("dayofyear", "tweets / day of year", tweets_df, size=3, ordered=False)

## Plot: Tweet/ Date

In [ ]:
plot_count("date_only", "tweets / date", tweets_df,size=4, ordered=False)

## PLot: tweets / hour

In [ ]:
plot_count("hour", "tweets / hour", tweets_df,size=4, ordered=False)

In [ ]:
plot_count("minute", "tweets / minute", tweets_df,size=5, ordered=False)

# Opinon and Sentiment Analysis- To do

In [ ]:
tweets_df.head(3)

## 1. Sentiment Analysis : NLTK VADER

In [ ]:
# from textblob import TextBlob
# def get_sentiment(text):
#     blob = TextBlob(text)
#     sentiment_polarity = blob.sentiment.polarity
#     sentiment_subjectivity = blob.sentiment.subjectivity
#     if sentiment_polarity > 0:
#         sentiment_label = 'Positive'
#     elif sentiment_polarity < 0:
#         sentiment_label = 'Negative'
#     else:
#         sentiment_label = 'Neutral'
#     result = {'polarity':sentiment_polarity,
#               'subjectivity':sentiment_subjectivity,
#               'sentiment':sentiment_label}
#     return result
# get_sentiment("I got tested positive after vaccination")

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [ ]:
# borrowed from https://www.kaggle.com/pashupatigupta/sentiments-transformer-vader-embedding-bert
# sia = SentimentIntensityAnalyzer()
def find_sentiment_score(post):
    return sia.polarity_scores(post)["compound"] 

In [ ]:
def find_sentiment(post):
    if sia.polarity_scores(post)["compound"] > 0:
        return "Positive"
    elif sia.polarity_scores(post)["compound"] < 0:
        return "Negative"
    else:
        return "Neutral" 

In [ ]:
find_sentiment_score("I got tested positive after vaccination")

In [ ]:
def plot_sentiment(df, feature, title):
    counts = df[feature].value_counts()
    percent = counts/sum(counts)

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

    counts.plot(kind='bar', ax=ax1, color='green')
    percent.plot(kind='bar', ax=ax2, color='blue')
    ax1.set_ylabel(f'Counts : {title} sentiments', size=12)
    ax2.set_ylabel(f'Percentage : {title} sentiments', size=12)
    plt.suptitle(f"Sentiment analysis: {title}")
    plt.tight_layout()
    plt.show()

In [ ]:
tweets_df['sentiment_score'] = tweets_df['text'].apply(lambda x: find_sentiment_score(x))
tweets_df['sentiment'] = tweets_df['text'].apply(lambda x: find_sentiment(x))
plot_sentiment(tweets_df, 'sentiment', 'Text')

### Positive Sentiment

In [ ]:
us_df = tweets_df.loc[tweets_df.user_location=="United States"]
show_wordcloud(us_df.loc[us_df['sentiment']=='Positive', 'text'], title = 'Prevalent words in tweets (Positive sentiment)')

### Negative Sentiment

In [ ]:
show_wordcloud(tweets_df.loc[tweets_df['sentiment']=='Negative', 'text'], title = 'Prevalent words in tweets (Negative sentiment)')

### Neutral Sentiment

In [ ]:
show_wordcloud(tweets_df.loc[tweets_df['sentiment']=='Neutral', 'text'], title = 'Prevalent words in tweets (Neutral sentiment)')

# Assigning Polarity & Subjectivity to tweets

In [ ]:
tweets_df.columns

In [ ]:
tweets_df.head(2)

In [ ]:
# Cleaning the data using neattext library
tweets_df['clean_data']=tweets_df['text'].apply(ntx.remove_hashtags)
tweets_df['clean_data']=tweets_df['clean_data'].apply(ntx.remove_urls)
tweets_df['clean_data']=tweets_df['clean_data'].apply(ntx.remove_userhandles)
tweets_df['clean_data']=tweets_df['clean_data'].apply(ntx.remove_multiple_spaces)
tweets_df['clean_data']=tweets_df['clean_data'].apply(ntx.remove_special_characters)

In [ ]:
tweets_df[['clean_data','text']].head()

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
#stopwords are the words which won't bring about any changes to the polarity of the tweet
stop_words = stopwords.words('english')   
len(stop_words),stop_words[5:10]

In [ ]:
# function to remove stopwords
def stopWords(tweet):
  clean_tweet = tweet
  clean_tweet = " ".join(word for word in clean_tweet.split() if word not in stop_words)
# clean_tweet = " ".join(lemmatizer.lemmatize(word) for word in clean_tweet.split())
  return clean_tweet

In [ ]:
 tweets_df['clean_data'] = tweets_df['clean_data'].apply(lambda x: stopWords(x))

In [ ]:
tweets_df.head(2)

In [ ]:
from textblob import TextBlob
# Function to assign polarity and subjectivity to the tweets
def blob_fun(text):
  senti = TextBlob(text)
  senti_polarity = senti.sentiment.polarity
  senti_subjectivity = senti.sentiment.subjectivity

  if senti_polarity > 0:
    res = 'Positive'

  elif senti_polarity < 0:
    res = 'Negative'

  elif senti_polarity == 0:
    res ="Neutral"

  result = {'polarity':senti_polarity,'subjectivity':senti_subjectivity,'sentiment':res}

  return result

In [ ]:
blob_fun(tweets_df['clean_data'][5])

In [ ]:
# but this isn't always right as shown in the example below
blob_fun('thank god,i tested negative for covid')

In [ ]:
tweets_df['results'] = tweets_df['clean_data'].apply(blob_fun)

In [ ]:
tweets_df['results'].head()

In [ ]:
# tweets_df.drop(columns={"user_location",'text'},inplace=True)

In [ ]:
tweets_df.head(2)

In [ ]:
tweets_df = tweets_df.join(pd.json_normalize(data=tweets_df['results']))

In [ ]:
tweets_df.head(2)

In [ ]:
# categorized tweets in seperate Series
positive_tweet =  tweets_df[tweets_df['sentiment'] == 'Positive']['clean_data']
negative_tweet =  tweets_df[tweets_df['sentiment'] == 'Negative']['clean_data']
neutral_tweet =  tweets_df[tweets_df['sentiment'] == 'Neutral']['clean_data']

In [ ]:
from wordcloud import WordCloud
# Function for creating WordClouds
def cloud_of_Words(tweet_cat,title):
    forcloud = ' '.join([tweet for tweet in tweet_cat])
    wordcloud = WordCloud(width =500,height = 300,random_state =5,max_font_size=110).generate(forcloud)
    plt.imshow(wordcloud, interpolation ='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()
    plt.figure(figsize = (10,8))

In [ ]:
plt.figure(figsize = (10,8))
# Creating wordclouds for positive, negative, neutral tweets
cloud_of_Words(positive_tweet, 'Positive')
cloud_of_Words(negative_tweet, 'Negative')
cloud_of_Words(neutral_tweet, 'Neutral')

In [ ]:
#The above three wordclouds have the similar words as expected because our main analysis is Covid Vaccine

# Breaking down the tweets into words in seperate categories
positive_tokens = [token for line in positive_tweet for token in line.split()]
negative_tokens = [token for line in negative_tweet for token in line.split()]
neutral_tokens = [token for line in neutral_tweet for token in line.split()]

In [ ]:
from collections import Counter
# to get most used words
def get_maxtoken(tweets,num=30):
  word_tokens = Counter(tweets)
  max_common = word_tokens.most_common(num)
  return dict(max_common)

In [ ]:
def token_df_vis(x, title):
  df = pd.DataFrame(get_maxtoken(x).items(),columns=['words','count'])
  # plt.figure(figsize = (20,5))
  # plt.title(title)
  # plt.xticks(rotation=45)
  fig = px.bar(df,x='words',y='count',title = title)
  fig.show()

In [ ]:
token_df_vis(positive_tokens,'Positive')
token_df_vis(negative_tokens,'Negative')
token_df_vis(neutral_tokens,'Neutral')

## the spread of our tweets on polarity vs subjectivity

In [ ]:
fig = px.scatter(tweets_df,x='polarity',y='subjectivity')
fig.show()

# Vaccine wise Analysis

In [ ]:
deep = tweets_df.drop(columns="results")
deep.head(2)

## Reference Tag for 5 Vaccines

In [ ]:
# creating reference tags for 5 vaccines -> Pfizer, Covaxin(Bharat Biotech), Sputnik,AstraZenca(Covishield),Moderna
pfizer_refs = ["Pfizer","pfizer","Pfizer–BioNTech","pfizer-bioNtech","BioNTech","biontech","PfizerBioNTech"]
bbiotech_refs = ["covax","covaxin","Covax","Covaxin","Bharat Biotech","bharat biotech","BharatBiotech","bharatbiotech"]
sputnik_refs = ["russia","sputnik","Sputnik","V"]
astra_refs = ['sii','SII','adar poonawalla','Covishield','covishield','astra','zenca','Oxford–AstraZeneca','astrazenca','oxford-astrazenca','serum institiute']
moderna_refs = ['moderna','Moderna','mRNA-1273','Spikevax']

In [ ]:
def refer(tweet, refs):
  flag =0
  for ref in refs:
    if tweet.find(ref) != -1:
      flag =1
  return flag

deep['pfizer'] = deep['clean_data'].apply(lambda x : refer(x, pfizer_refs))
deep['bbiotech'] = deep['clean_data'].apply(lambda x : refer(x, bbiotech_refs))
deep['sputnik'] = deep['clean_data'].apply(lambda x : refer(x, sputnik_refs))
deep['astra'] = deep['clean_data'].apply(lambda x : refer(x, astra_refs))
deep['moderna'] = deep['clean_data'].apply(lambda x : refer(x, moderna_refs))

In [ ]:
display(deep.pfizer.value_counts(),deep.bbiotech.value_counts(),deep.sputnik.value_counts(),deep.astra.value_counts(),deep.moderna.value_counts())

In [ ]:
deep[deep['pfizer']==1].head(2)    #what the dataset looks like

In [ ]:
import plotly.graph_objects as go
def plotAllSentimentGraph(vac_type):
  df_va = deep[deep[vac_type]==1][['date_only','sentiment']]
  df_vd = df_v.groupby(df_va['date_only'])['sentiment'].value_counts().reset_index(name='counts')
  df_vmp = df_vd[df_vd['sentiment']=='Positive']
  df_vmn = df_vd[df_vd['sentiment']=='Negative']
  df_vmnu = df_vd[df_vd['sentiment']=='Neutral']
  fig = go.Figure()
  fig.add_trace(
    go.Scatter(
        x=df_vmp['date_only'],
        y=df_vmp['counts'],
        name = "Positive Tweets"
    ))
  fig.add_trace(
    go.Scatter(
        x=df_vmn['date_only'],
        y=df_vmn['counts'],
        name = "Negative Tweets"
    ))
  fig.add_trace(
    go.Scatter(
        x=df_vmnu['date_only'],
        y=df_vmnu['counts'],
        name = "Neutral Tweets"
    ))
  fig.show()

plotAllSentimentGraph('pfizer')

In [ ]:
plotAllSentimentGraph('bbiotech')

In [ ]:
plotAllSentimentGraph('astra')

In [ ]:
plotAllSentimentGraph('moderna')

In [ ]:
plotAllSentimentGraph('sputnik')

In [ ]:
v1 = pd.DataFrame(deep['astra'].value_counts().reset_index(name='counts'))
v1 = v1[v1['index'] != 0]
v1['vaccine'] = 'astra'

v2 = pd.DataFrame(deep['bbiotech'].value_counts().reset_index(name='counts'))
v2 = v2[v2['index'] != 0]
v2['vaccine'] = 'bbiotech'

v3 = pd.DataFrame(deep['moderna'].value_counts().reset_index(name='counts'))
v3 = v3[v3['index'] != 0]
v3['vaccine'] = 'moderna'

v4 = pd.DataFrame(deep['pfizer'].value_counts().reset_index(name='counts'))
v4 = v4[v4['index'] != 0]
v4['vaccine'] = 'pfizer'

v5 = pd.DataFrame(deep['sputnik'].value_counts().reset_index(name='counts'))
v5 = v5[v5['index'] != 0]
v5['vaccine'] = 'sputnik'

frames = [v1, v2, v3, v4, v5]
result = pd.concat(frames)
result.head()

In [ ]:
data = {
   "values": result['counts'][:15],
   "labels": result['vaccine'],
   "domain": {"column": 0},
   "name": "Vaccine Name",
   "hoverinfo":"label+percent+name",
   "hole": .4,
   "type": "pie"
}
layout = go.Layout(title="<b>Ratio on Vaccine</b>", legend=dict(x=0.1, y=1.1, orientation="h"))

data = [data]
fig = go.Figure(data = data, layout = layout)
fig.update_layout(title_x=0.5)
fig.show()

## MonthWise Vaccine Analysis

In [ ]:
def vaccineMonthwiseAnalysis(vac_type,mon_num):
  df_v = deep[deep[vac_type]==1][['month','sentiment']]
  df_vm = df_v.groupby(df_v['month'])['sentiment'].value_counts().reset_index(name='counts')
  df_vma = df_vm[df_vm['month']==mon_num]
  fig = px.bar(df_vma, x='sentiment', y='counts',color="sentiment")
  fig.show()
  return df_vma.head()

In [ ]:
vaccineMonthwiseAnalysis('sputnik',1)

## Polarity and Subjectivity of Different Vaccines

In [ ]:
def stats(a,b,c,d,e):
  for i in a,b,c,d,e:
     display(deep[deep[i]==1][[i,'polarity','subjectivity']].groupby(i).agg([np.mean,np.max,np.min,np.median]))

In [ ]:
stats('pfizer','bbiotech','sputnik','astra','moderna')

In [ ]:
pfizer = deep[deep['pfizer']==1][['date','polarity']]
bbiotech = deep[deep['bbiotech']==1][['date','polarity']]
sputnik = deep[deep['sputnik']==1][['date','polarity']]
astra = deep[deep['astra']==1][['date','polarity']]
moderna = deep[deep['moderna']==1][['date','polarity']]

pfizer = pfizer.sort_values(by='date',ascending=True)
bbiotech = bbiotech.sort_values(by='date',ascending=True)
sputnik = sputnik.sort_values(by='date',ascending=True)
astra = astra.sort_values(by='date',ascending=True)
moderna = moderna.sort_values(by='date',ascending=True)

pfizer['Avg Polarity'] = pfizer.polarity.rolling(20, min_periods=3).mean()
bbiotech['Avg Polarity'] = bbiotech.polarity.rolling(20, min_periods=3).mean()
sputnik['Avg Polarity'] = sputnik.polarity.rolling(20, min_periods=3).mean()
astra['Avg Polarity'] = astra.polarity.rolling(5, min_periods=3).mean()
moderna['Avg Polarity'] = moderna.polarity.rolling(20, min_periods=3).mean()

In [ ]:
pfizer.head(50)

# Retweet Count Vs Polarity Over Timeline

In [ ]:
timeline = tweets_df.groupby(['date']).agg(np.nanmean).reset_index()
timeline['count'] = tweets_df.groupby(['date']).count().reset_index()['retweets']
timeline = timeline[['date', 'count', 'polarity', 'retweets', 'favorites', 'subjectivity']]
timeline["polarity"] = timeline["polarity"].astype(float)
timeline["subjectivity"] = timeline["subjectivity"].astype(float)
timeline.head(2)

In [ ]:
fig = px.line(timeline, x='date', y='count')
fig.show()


# Visualizing the vaccine Polarity (Moving Average) vs Time

In [ ]:
tweets_df.head(2)

In [ ]:
fig = px.line(pfizer, x='date', y='polarity')
fig.show()

In [ ]:
a,b,c,d,e = pfizer,bbiotech,sputnik,astra,moderna
fig = px.line(a, x="date", y="Avg Polarity", title='Pfizer')
fig.show()
fig = px.line(b, x="date", y="Avg Polarity", title='Bharat Biotech')
fig.show()
fig = px.line(c, x="date", y="Avg Polarity", title='Sputnik')
fig.show()
fig = px.line(d, x="date", y="Avg Polarity", title='AstraZence/Covishield')
fig.show()
fig = px.line(e, x="date", y="Avg Polarity", title='Moderna')
fig.show()

# Polarity and Average polarity for All Vaccines

In [ ]:
import plotly.graph_objects as go

def plotPolarityGraph(vac_type):
  fig = go.Figure()
  fig.add_trace(
    go.Scatter(
        x=vac_type['date'],
        y=vac_type['polarity'],
        name = "Complete Polarity"
    ))
  fig.add_trace(
    go.Scatter(
        x=vac_type['date'],
        y=vac_type['Avg Polarity'],
        name = "Average Polarity"
    ))
  fig.show()

In [ ]:
plotPolarityGraph(bbiotech)

In [ ]:
plotPolarityGraph(sputnik)

In [ ]:

plotPolarityGraph(astra)

In [ ]:
plotPolarityGraph(moderna)

In [ ]:
plotPolarityGraph(pfizer)

In [ ]:
loc_analysis = pd.DataFrame(df_vacc_tweet['user_location'].value_counts().sort_values(ascending=False))
loc_analysis = loc_analysis.rename(columns={'user_location':'count'})

data = {
   "values": loc_analysis['count'][:15],
   "labels": loc_analysis.index[:15],
   "domain": {"column": 0},
   "name": "Location Name",
   "hoverinfo":"label+percent+name",
   "hole": .4,
   "type": "pie"
}
layout = go.Layout(title="<b>Ratio on Location</b>", legend=dict(x=0.1, y=1.1, orientation="h"))

data = [data]
fig = go.Figure(data = data, layout = layout)
fig.update_layout(title_x=0.5)
fig.show()

# Visualizing Overall Vaccine Polarity

In [ ]:
total=pd.DataFrame()
total['date'] = sorted(deep['date'].unique())
senti=list()
for date in total['date']:
    senti.append(deep[deep['date']==date].polarity.mean())
total['Sentiment']=senti
fig = px.line(total, x="date", y="Sentiment", title='Overall Sentiment around Vaccines')
fig.show() 

# References  
* 1. https://www.kaggle.com/amartyanambiar/covid-19-vaccine-sentitmental-analysis/notebook (used upto line 15)
* 2. https://www.kaggle.com/gpreda/explore-vaccines-tweets#Data-preparation (up to before Sentiment analysis, but have to fix time distribution plots)
* 3. https://towardsdatascience.com/sentiment-analysis-of-covid-19-vaccine-tweets-dc6f41a5e1af
------------
2nd Dataset

1. https://www.kaggle.com/filterjoe/world-covid-19-vaccines-complex-visualizations
2. https://www.kaggle.com/aliderakhshesh/time-series-with-ml